<a href="https://colab.research.google.com/github/yapyang040527/PLanguage/blob/main/HW3_%E5%BE%85%E8%BE%A6%E6%B8%85%E5%96%AE%E8%88%87%E7%95%AA%E8%8C%84%E9%90%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil

In [ ]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [ ]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1J7hscBdnKmeBaSiSA9z0_6XbcCnPKxj5DzCIfxyLR-g/edit?gid=84564613#gid=84564613"
WORKSHEET_NAME = "工作表1"
TIMEZONE = "Asia/Taipei"

In [ ]:
import pandas as pd
from google.colab import auth
from google.auth import default
import gspread

# Authenticate and authorize Google Sheets access
# This code is repeated here to ensure gc is defined even if the previous cell failed.
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("Google Sheets authentication successful.")
except Exception as e:
    print(f"Google Sheets authentication failed: {e}")
    # Depending on the severity, you might want to exit or handle this differently
    # For now, we'll print the error and continue, but subsequent operations might fail.
    gc = None # Ensure gc is None if authentication fails


# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
if gc is not None:
    try:
        gsheets = gc.open_by_url(SHEET_URL)

        # 從 gsheets 的 All-whiteboard-device 載入 sheets
        sh = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
        # 將 sheets1 資料載入 pd 的 DataFrame 進行分析
        df = pd.DataFrame(sh[1:], columns=sh[0])
        # 取得最前面的5筆資料
        display(df.head())
    except Exception as e:
        print(f"Error accessing Google Sheet: {e}")
else:
    print("Google Sheets client (gc) is not initialized due to authentication failure. Cannot read data.")

Google Sheets authentication successful.


""


In [ ]:
import pandas as pd
from google.colab import auth
from google.auth import default
import gspread

# 確保認證物件存在，初始化為 None
gc = None

def authenticate_sheets():
    """嘗試進行 Google Sheets 互動式認證"""
    global gc
    try:
        # 嘗試使用 auth.authenticate_user() 進行認證
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        print("✅ Google Sheets 認證成功。")
        return True
    except Exception as e:
        print(f"❌ Google Sheets 認證失敗：{e}")
        # 不使用 sys.exit()，而是返回 False，讓程式繼續執行，但會跳過 Sheets 相關操作
        return False

# 執行認證
if not authenticate_sheets():
    print("⚠️ 認證失敗，Sheets 相關功能將無法使用。請檢查您的瀏覽器彈窗或 Colab 連線狀態。")

# --- 接下來的函式只有在認證成功時才執行 ---

def ensure_spreadsheet(name):
    """確保試算表存在，若不存在則創建。"""
    if gc is None:
        print("❌ 無法執行 ensure_spreadsheet：尚未成功授權 Gspread。")
        return None
    try:
        sh = gc.open(name)  # returns gspread.models.Spreadsheet
    except gspread.SpreadsheetNotFound:
        print(f"找不到試算表 '{name}'，正在創建...")
        sh = gc.create(name)
    return sh

# --- 範例使用 (假設 WORKSHEET_NAME 已定義) ---
if gc is not None:
    # ⚠️ 請確保 WORKSHEET_NAME 在前面的 Cell 中已被定義，否則這裡會出現 NameError
    try:
        WORKSHEET_NAME = "My_Todo_Pomodoro_Data"  # 範例定義
        sh = ensure_spreadsheet(WORKSHEET_NAME)
        if sh:
            print(f"✅ 成功確保試算表 '{WORKSHEET_NAME}' 存在。")
    except NameError:
        print("❌ 錯誤：WORKSHEET_NAME 變數尚未定義。")
    except Exception as e:
        print(f"❌ 確保試算表時發生錯誤: {e}")

✅ Google Sheets 認證成功。
找不到試算表 'My_Todo_Pomodoro_Data'，正在創建...
❌ 確保試算表時發生錯誤: APIError: [403]: User rate limit exceeded.


In [ ]:
# 完整版：番茄主題任務儀表板（Colab + Google Sheets + 動畫計時器）
# 請在 Colab 環境中執行；若尚未授權 Google，會在執行時要求授權。

import pandas as pd
import gradio as gr
from google.colab import auth
from google.auth import default
import gspread
from gspread_dataframe import get_as_dataframe
import uuid
from datetime import datetime as dt
from dateutil.tz import gettz
import requests
from bs4 import BeautifulSoup
import json
import sys
import io
import tempfile
import matplotlib.pyplot as plt
from dateutil import tz
import datetime # 引入 datetime 模組用於日期處理

# ------------------------
# 設定（如需可在執行 cell 前先定義 SHEET_URL / WORKSHEET_NAME / TIMEZONE）
# ------------------------
try:
    SHEET_URL
except NameError:
    SHEET_URL = None  # 若有 sheet url 可直接設定
try:
    WORKSHEET_NAME
except NameError:
    WORKSHEET_NAME = "TasksSheet"
try:
    TIMEZONE
except NameError:
    TIMEZONE = "Asia/Taipei"

def tznow():
    return dt.now(gettz(TIMEZONE))

# --- Google Sheets auth ---
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("Google Sheets authentication successful.")
except Exception as e:
    print("Google Sheets authentication failed or skipped:", e)
    gc = None

def ensure_spreadsheet(url):
    if gc is None:
        return None
    try:
        if url:
            sh = gc.open_by_url(url)
        else:
            sh = gc.open(WORKSHEET_NAME)
        return sh
    except Exception:
        try:
            import re
            if url:
                m = re.search(r'/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
                if m:
                    return gc.open_by_key(m.group(1))
            return gc.open(WORKSHEET_NAME)
        except Exception as e:
            print("Could not open spreadsheet:", e)
            return None

sh = ensure_spreadsheet(SHEET_URL)

# Headers
TASKS_HEADER = [
    "id","task","status","priority","est_min","start_time","end_time",
    "actual_min","pomodoros","due_date","labels","notes",
    "created_at","updated_at","completed_at","planned_for"
]
LOGS_HEADER = ["log_id","task_id","phase","start_ts","end_ts","minutes","cycles","note"]
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_task"]

def ensure_worksheet(sh, title, header):
    if sh is None: return None
    try:
        ws = sh.worksheet(title)
    except Exception:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])
    # ensure header
    try:
        data = ws.get_all_values()
        if not data or data[0] != header:
            ws.clear()
            ws.update([header])
    except Exception:
        pass
    return ws

if sh:
    ws_tasks = ensure_worksheet(sh, "tasks", TASKS_HEADER)
    ws_logs  = ensure_worksheet(sh, "pomodoro_logs", LOGS_HEADER)
    ws_clips = ensure_worksheet(sh, "web_clips", CLIPS_HEADER)
else:
    ws_tasks = ws_logs = ws_clips = None
    print("Running without Google Sheets connection. Using local DataFrame until sheet available.")

# read/write helpers
def read_df(ws, header):
    if ws is None:
        return pd.DataFrame(columns=header)
    try:
        df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    except Exception:
        df = pd.DataFrame()
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    for c in header:
        if c not in df.columns:
            df[c] = ""
    for numcol in ("est_min","actual_min","pomodoros"):
        if numcol in df.columns:
            df[numcol] = pd.to_numeric(df[numcol], errors="coerce").fillna(0).astype(int)
    return df[header]

def write_df(ws, df, header):
    if ws is None:
        return
    if df is None or df.empty:
        ws.clear()
        ws.update([header])
        return
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)
    ws.clear()
    ws.update([header] + df_out.values.tolist())

# load initial
try:
    tasks_df = read_df(ws_tasks, TASKS_HEADER)
    logs_df = read_df(ws_logs, LOGS_HEADER)
    clips_df = read_df(ws_clips, CLIPS_HEADER)
except Exception as e:
    print("Initial load failed:", e)
    tasks_df = pd.DataFrame(columns=TASKS_HEADER)
    logs_df = pd.DataFrame(columns=LOGS_HEADER)
    clips_df = pd.DataFrame(columns=CLIPS_HEADER)

# core logic
_active_sessions = {}

def recalc_task_actuals(task_id):
    global tasks_df, logs_df
    work_logs = logs_df[(logs_df["task_id"]==task_id) & (logs_df["phase"]=="work")]
    total_min = work_logs["minutes"].astype(float).sum() if not work_logs.empty else 0
    pomos = int(round(total_min / 25.0))
    idx = tasks_df.index[tasks_df["id"]==task_id]
    if len(idx)==0: return
    i = idx[0]
    tasks_df.loc[i,"actual_min"] = int(total_min)
    tasks_df.loc[i,"pomodoros"] = pomos
    tasks_df.loc[i,"updated_at"] = tznow().isoformat()

def list_task_choices():
    global tasks_df
    if tasks_df.empty:
        return []
    choices = []
    # 優先顯示 todo 和 in-progress 的任務
    priority_order = ["in-progress", "todo", "done"]
    tasks_df['sort_key'] = tasks_df['status'].apply(lambda x: priority_order.index(x) if x in priority_order else 99)
    sorted_df = tasks_df.sort_values(by=['sort_key', 'priority'], ascending=[True, False])

    for _, r in sorted_df.iterrows():
        label = f"[{r['status']}] (P:{r['priority']}) {r['task']} — {r['id']}"
        choices.append((label, r["id"]))

    tasks_df.drop(columns=['sort_key'], errors='ignore', inplace=True)
    return choices

# CRUD and utilities
def add_task(task, priority, est_min, due_date_str, labels, notes, planned_for):
    global tasks_df
    _now = tznow().isoformat()

    # 修正: 由於無法使用日期選擇器，due_date_str 保持為字串，由使用者手動輸入
    due_date_str = due_date_str or "" # 確保是字串

    new = pd.DataFrame([{
        "id": str(uuid.uuid4())[:8],
        "task": (task or "").strip(),
        "status": "todo",
        "priority": priority or "M",
        "est_min": int(est_min) if est_min else 25,
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "pomodoros": 0,
        "due_date": due_date_str, # 使用者手動輸入的字串
        "labels": labels or "",
        "notes": notes or "",
        "created_at": _now,
        "updated_at": _now,
        "completed_at": "",
        "planned_for": planned_for or ""
    }])
    tasks_df = pd.concat([tasks_df, new], ignore_index=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    # 返回任務列表 Dataframe 和新的選擇列表
    return "✅ 已新增任務", tasks_df, list_task_choices(), list_task_choices()

def delete_task(task_id):
    global tasks_df
    if not task_id:
        return "⚠️ 請提供要刪除的任務 ID", tasks_df, list_task_choices(), list_task_choices()
    idx = tasks_df.index[tasks_df["id"]==task_id]
    if len(idx)==0:
        return "⚠️ 找不到任務", tasks_df, list_task_choices(), list_task_choices()
    tasks_df = tasks_df.drop(idx)
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "🗑️ 任務已刪除", tasks_df, list_task_choices(), list_task_choices()

def update_task_status(task_id, new_status):
    global tasks_df
    idx = tasks_df.index[tasks_df["id"]==task_id]
    if len(idx)==0:
        return "⚠️ 找不到任務", tasks_df, list_task_choices(), list_task_choices()
    i = idx[0]
    tasks_df.loc[i,"status"] = new_status
    tasks_df.loc[i,"updated_at"] = tznow().isoformat()
    if new_status=="done" and not tasks_df.loc[i,"completed_at"]:
        tasks_df.loc[i,"completed_at"] = tznow().isoformat()
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "✅ 狀態已更新", tasks_df, list_task_choices(), list_task_choices()

# Pomodoro backend hooks that also return JS to control client timer
def start_phase(task_id, phase, cycles):
    if not task_id:
        return "⚠️ 請先選擇任務", ""
    _active_sessions[task_id] = {
        "phase": phase,
        "start_ts": tznow().isoformat(),
        "cycles": int(cycles) if cycles else 1
    }
    # return message + JS snippet to start client timer (seconds)
    seconds = 25*60 if phase=="work" else 5*60
    # JS will call window.startTimer(seconds)
    script = f"<script>if(window.startTimer){{window.startTimer({seconds});}}else{{console.warn('timer not found');}}</script>"
    return f"▶️ 已開始：{phase}（task: {task_id}）", script

def end_phase(task_id, note):
    global logs_df, tasks_df
    if task_id not in _active_sessions:
        # still instruct client to stop timer if present
        return "⚠️ 尚未開始任何階段", "<script>if(window.stopTimer)window.stopTimer();</script>"
    sess = _active_sessions.pop(task_id)
    start = pd.to_datetime(sess["start_ts"])
    end = tznow()
    minutes = round((end - start).total_seconds() / 60.0, 2)
    log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8],
        "task_id": task_id,
        "phase": sess["phase"],
        "start_ts": start.isoformat(),
        "end_ts": end.isoformat(),
        "minutes": minutes,
        "cycles": int(sess["cycles"]),
        "note": note or ""
    }])
    logs_df = pd.concat([logs_df, log], ignore_index=True)
    write_df(ws_logs, logs_df, LOGS_HEADER)
    if sess["phase"] == "work":
        recalc_task_actuals(task_id)
        write_df(ws_tasks, tasks_df, TASKS_HEADER)
    # also stop client timer
    script = "<script>if(window.stopTimer)window.stopTimer();</script>"
    return f"⏹️ 已結束：{sess['phase']}，紀錄 {minutes} 分鐘", script

# Crawler
def crawl(url, selector, mode, limit):
    try:
        resp = requests.get(url, timeout=15, headers={"User-Agent":"Mozilla/5.0"})
        resp.raise_for_status()
    except Exception as e:
        # Gradio 爬蟲結果輸出是 (df, msg)，這裡需返回空 df
        return pd.DataFrame(columns=CLIPS_HEADER), f"⚠️ 請求失敗：{e}"
    soup = BeautifulSoup(resp.text, "html.parser")
    nodes = soup.select(selector)
    rows = []
    for i, n in enumerate(nodes[:int(limit) if limit else 20]):
        text = n.get_text(strip=True) if mode in ("text","both") else ""
        href = n.get("href") if mode in ("href","both") else ""
        if href and href.startswith("/"):
            from urllib.parse import urljoin
            href = urljoin(url, href)
        rows.append({
            "clip_id": str(uuid.uuid4())[:8],
            "url": url,
            "selector": selector,
            "text": text,
            "href": href,
            "created_at": tznow().isoformat(),
            "added_to_task": ""
        })
    df = pd.DataFrame(rows, columns=CLIPS_HEADER)
    return df, f"✅ 擷取 {len(df)} 筆"

def add_clips_as_tasks(clip_ids, default_priority, est_min):
    global clips_df, tasks_df
    # 這裡 clip_ids 是一個逗號分隔的字串
    if not clip_ids:
        return "⚠️ 請先勾選要加入的爬蟲項目", clips_df, tasks_df, list_task_choices(), list_task_choices()

    # 解析 clip_ids 列表
    id_list = [i.strip() for i in (clip_ids or "").split(",") if i.strip()]
    sel = clips_df[clips_df["clip_id"].isin(id_list)]
    _now = tznow().isoformat()
    new_tasks = []

    for _, r in sel.iterrows():
        title = r["text"] or r["href"] or "（未命名）"
        note = f"來源：{r['url']}\n選擇器：{r['selector']}\n連結：{r['href']}"
        new_tasks.append({
            "id": str(uuid.uuid4())[:8],
            "task": title[:120],
            "status": "todo",
            "priority": default_priority or "M",
            "est_min": int(est_min) if est_min else 25,
            "start_time": "",
            "end_time": "",
            "actual_min": 0,
            "pomodoros": 0,
            "due_date": "",
            "labels": "from:crawler",
            "notes": note,
            "created_at": _now,
            "updated_at": _now,
            "completed_at": "",
            "planned_for": ""
        })
    if new_tasks:
        tasks_df = pd.concat([tasks_df, pd.DataFrame(new_tasks)], ignore_index=True)
        clips_df.loc[clips_df["clip_id"].isin(id_list), "added_to_task"] = "yes"
        write_df(ws_tasks, tasks_df, TASKS_HEADER)
        write_df(ws_clips, clips_df, CLIPS_HEADER)
        return f"✅ 已加入 {len(new_tasks)} 項為任務", clips_df, tasks_df, list_task_choices(), list_task_choices()
    return "⚠️ 無可加入項目", clips_df, tasks_df, list_task_choices(), list_task_choices()

# Export/Import
def export_tasks_csv():
    global tasks_df
    f = tempfile.NamedTemporaryFile(delete=False, suffix='.csv')
    tasks_df.to_csv(f.name, index=False)
    return f.name

def export_tasks_json():
    global tasks_df
    f = tempfile.NamedTemporaryFile(delete=False, suffix='.json')
    tasks_df.to_json(f.name, orient='records', force_ascii=False)
    return f.name

def import_tasks_file(file_obj):
    global tasks_df
    if file_obj is None:
        return "⚠️ 請上傳 CSV 或 JSON 檔案", tasks_df
    try:
        # 重設檔案指標到開頭
        file_obj.seek(0)
        content = file_obj.read()

        try:
            df = pd.read_csv(io.BytesIO(content))
        except Exception:
            # 如果不是 CSV，嘗試作為 JSON 讀取
            df = pd.read_json(io.BytesIO(content))

        # 確保匯入的 DataFrame 包含所有必需的欄位
        for c in TASKS_HEADER:
            if c not in df.columns:
                df[c] = ""

        df = df[TASKS_HEADER]
        tasks_df = df.copy()
        write_df(ws_tasks, tasks_df, TASKS_HEADER)
        return f"✅ 匯入成功，匯入 {len(tasks_df)} 筆", tasks_df
    except Exception as e:
        return f"⚠️ 匯入失敗：{e}", tasks_df

# Visualization
def generate_stats_image():
    global tasks_df
    # 確保中文字體顯示 (適用於 Colab 環境)
    plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'sans-serif']
    plt.rcParams['axes.unicode_minus'] = False # 解決負號亂碼

    fig, axes = plt.subplots(1,2,figsize=(10,4))

    if tasks_df is None or tasks_df.empty:
        axes[0].text(0.5,0.5,'No data', transform=axes[0].transAxes, ha='center')
        axes[1].text(0.5,0.5,'No data', transform=axes[1].transAxes, ha='center')
    else:
        # 狀態統計 (Status)
        status_counts = tasks_df['status'].value_counts()
        if not status_counts.empty:
            axes[0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%')
            axes[0].set_title('任務狀態分佈')
        else:
            axes[0].text(0.5,0.5,'No Status Data', transform=axes[0].transAxes, ha='center')

        # Top Pomodoros
        top = tasks_df[tasks_df['pomodoros']>0].sort_values('pomodoros', ascending=False).head(5)
        if not top.empty:
            axes[1].barh(top['task'].astype(str).str[:20], top['pomodoros']) # barh for better label visibility
            axes[1].tick_params(axis='y', rotation=0)
            axes[1].set_title('Top 5 番茄數任務')
            axes[1].set_xlabel('番茄數 (Pomodoros)')
        else:
            axes[1].text(0.5,0.5,'No Pomodoros Logged', transform=axes[1].transAxes, ha='center')

    plt.tight_layout()
    f = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    fig.savefig(f.name, bbox_inches='tight')
    plt.close(fig)
    return f.name

# === Client-side Timer HTML + JS (略) ===
TIMER_HTML = """
<style>
.tomato-timer { display:flex; align-items:center; justify-content:center; margin:10px; }
/* ... (略 - 保持原始 CSS) ... */
.timer-circle {
  width:220px;height:220px;border-radius:50%;
  background: conic-gradient(#e74c3c var(--pct,0%), #eee 0%);
  display:flex; align-items:center; justify-content:center; box-shadow:0 8px 24px rgba(231,76,60,0.14);
}
.timer-inner { width:84%; height:84%; border-radius:50%; background:#fff; display:flex;flex-direction:column;align-items:center;justify-content:center; }
.timer-time{ font-size:28px; font-weight:800; color:#222; }
.timer-label{ font-size:12px; color:#666; margin-top:6px; }
.timer-controls{ display:flex; gap:8px; justify-content:center; margin-top:10px; }
.t-btn{ background:linear-gradient(90deg,#ff6b4a,#e74c3c); color:white; padding:8px 12px; border-radius:8px; border:none; cursor:pointer; }
.s-btn{ background:#fff; border:1px solid #eee; padding:8px 12px; border-radius:8px; cursor:pointer; }
</style>

<div class="tomato-timer">
  <div>
    <div class="timer-circle" id="timer-circle" style="--pct:0%">
      <div class="timer-inner">
        <div class="timer-time" id="timer-display">25:00</div>
        <div class="timer-label" id="timer-label">Ready</div>
      </div>
    </div>
    <div class="timer-controls">
      <button class="t-btn" id="js-start-work">Start Work</button>
      <button class="s-btn" id="js-stop">Stop</button>
    </div>
  </div>
</div>

<script>
window._tomato = { timer: null, remaining: 1500, running:false, total:1500 };

function formatTime(s){
  const mm = Math.floor(s/60).toString().padStart(2,'0');
  const ss = Math.floor(s%60).toString().padStart(2,'0');
  return `${mm}:${ss}`;
}

window.updateTimerUI = function(){
  const pct = (window._tomato.total - window._tomato.remaining) / window._tomato.total * 100;
  document.getElementById('timer-circle').style.setProperty('--pct', pct + '%');
  document.getElementById('timer-display').innerText = formatTime(window._tomato.remaining);
  document.getElementById('timer-label').innerText = window._tomato.running ? 'Running' : 'Paused';
}

window.startTimer = function(seconds){
  // if seconds changed, set total
  window._tomato.total = seconds;
  window._tomato.remaining = seconds;
  if(window._tomato.timer) { clearInterval(window._tomato.timer); window._tomato.timer = null; }
  window._tomato.running = true;
  updateTimerUI();
  window._tomato.timer = setInterval(function(){
    if(!window._tomato.running) return;
    window._tomato.remaining -= 1;
    if(window._tomato.remaining <= 0){
      clearInterval(window._tomato.timer);
      window._tomato.running = false;
      window._tomato.remaining = 0;
      // bell
      try{ new Audio('https://actions.google.com/sounds/v1/alarms/alarm_clock.ogg').play(); }catch(e){}
    }
    updateTimerUI();
  }, 1000);
}

window.stopTimer = function(){
  if(window._tomato.timer) clearInterval(window._tomato.timer);
  window._tomato.running = false;
  window._tomato.remaining = window._tomato.total;
  updateTimerUI();
}

// bind local buttons for convenience
document.addEventListener('DOMContentLoaded', function(){
  const swork = document.getElementById('js-start-work');
  const stop = document.getElementById('js-stop');
  if(swork) swork.onclick = function(){ startTimer(25*60); };
  if(stop) stop.onclick = function(){ stopTimer(); };
});
</script>"""

# ==== Gradio UI ====
def _refresh_all():
    global tasks_df, logs_df, clips_df
    tasks_df = read_df(ws_tasks, TASKS_HEADER)
    logs_df = read_df(ws_logs, LOGS_HEADER)
    clips_df = read_df(ws_clips, CLIPS_HEADER)
    # 這裡的 list_task_choices() 呼叫了兩次，分別對應 demo.load 輸出中的 task_choice 和 sel_task
    return tasks_df, logs_df, clips_df, list_task_choices(), today_summary(), list_task_choices()

def today_summary():
    global tasks_df
    today = tznow().date().isoformat()
    planned = tasks_df[((tasks_df['due_date']==today) | (tasks_df['planned_for'].str.lower()=='today'))]
    done = planned[planned['status']=='done']
    total = len(planned); done_n = len(done)
    rate = (done_n/total*100) if total>0 else 0
    return f"📅 今日計畫任務：{total}；✅ 完成：{done_n}；📈 完成率：{rate:.1f}%"

# 修正 1: 使用 theme='soft' 並允許切換黑暗模式
with gr.Blocks(
    title="番茄主題任務儀表板 (Colab)",
    theme=gr.themes.Soft(primary_hue="red", secondary_hue="gray"),
    css=None
) as demo:
    # 修正 2: 將元件實例移到 Blocks 內部頂部
    tasks_df_out = gr.Dataframe(value=tasks_df, headers=TASKS_HEADER, label="任務列表", interactive=True)
    logs_df_out = gr.Dataframe(value=logs_df, headers=LOGS_HEADER, label="計時記錄", interactive=False)
    clips_df_out = gr.Dataframe(value=clips_df, headers=CLIPS_HEADER, label="擷取項目", interactive=True)

    gr.Markdown("<div style='display:flex;align-items:center;gap:10px'><div style='width:48px;height:48px;border-radius:8px;background:linear-gradient(90deg,#ff6b4a,#e74c3c);display:flex;align-items:center;justify-content:center;color:white;font-weight:700'>🍅</div><div><h3 style='margin:0'>Tomato Tasks</h3><div style='color:#666;font-size:12px'>專注你的任務 · Google Sheet 同步</div></div></div>")

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### Tasks 📝 任務管理（新增 / 更新 / 刪除）")
            gr.Markdown("---")
            gr.Markdown("**🎯 新增任務：** 盡量使用**下拉選單**來設置優先級和規劃日，減少手動輸入。")
            task_in = gr.Textbox(label="任務名稱", placeholder="例如：寫報告 / 修 BUG / …")
            notes_in = gr.Textbox(label="備註（可選）")
            labels_state = gr.State(value="")

            with gr.Row():
                priority_in = gr.Dropdown(['H','M','L'], value='M', label='優先級 (H/M/L)')
                est_in = gr.Number(value=25, label='估時 (分鐘)', precision=0)
            with gr.Row():
                # 最終修正: 改回使用 Textbox，並提示使用者輸入格式
                due_in = gr.Textbox(label='到期日 (YYYY-MM-DD，手動輸入)', placeholder="例如：2025-12-31")
                planned_in = gr.Dropdown(['','today','tomorrow'], value='today', label='規劃 (快速選擇)')
            add_msg = gr.Markdown()
            add_btn = gr.Button("➕ 新增任務")
            gr.HTML("<hr>")
            gr.Markdown("**🔄 更新 / 刪除任務：** 從下拉選單中**選擇**任務 ID 和名稱，再進行操作。")
            task_choice = gr.Dropdown(choices=list_task_choices(), label='選取任務 (更新/刪除)')
            status_in = gr.Dropdown(['todo','in-progress','done'], value='in-progress', label='狀態')
            update_msg = gr.Markdown()
            update_btn = gr.Button("✏️ 更新狀態")
            delete_btn = gr.Button("🗑️ 刪除任務")
            gr.HTML("<hr>")
            gr.Markdown("### Import / Export 📤 匯入 / 匯出（與 Google Sheets 同步）")
            gr.Markdown("**⚠️ 注意：** 上傳檔案將會**覆蓋**當前的任務列表。")
            import_file = gr.File(label='上傳 CSV 或 JSON')
            import_msg = gr.Markdown()
            import_btn = gr.Button("⬆️ 匯入")
            with gr.Row():
                export_csv_btn = gr.Button("⬇️ 匯出 CSV")
                export_json_btn = gr.Button("⬇️ 匯出 JSON")
            file_out = gr.File()
            gr.HTML("<hr>")
            gr.Markdown("### Crawler 🕷️ 網頁擷取（快速生成任務）")
            gr.Markdown("**💡 使用方式：** 輸入網址和 **CSS 選擇器** (如 `a.post-title` 或 `li:nth-child(n)` )，點擊擷取，然後將結果的 **clip_id** 填入下方，即可批量生成任務。")
            url_in = gr.Textbox(label="目標 URL", placeholder="https://example.com/blog")
            selector_in = gr.Textbox(label="CSS selector", placeholder="a.news-item")
            mode_in = gr.Radio(["text","href","both"], value="text", label="擷取內容")
            limit_in = gr.Number(value=20, precision=0, label="最多擷取幾筆")
            crawl_btn = gr.Button("🕷️ 開始擷取")
            crawl_msg = gr.Markdown()
            gr.Markdown("---")
            gr.Markdown("**批量新增：** 將下方擷取結果表中的 **clip_id** 複製並用**逗號**分隔，即可新增為 `todo` 任務。")
            clip_ids_in = gr.Textbox(label="要加入任務的 clip_id (逗號分隔)")
            add_clips_btn = gr.Button("➕ 將擷取項目加入任務")
            add_clips_msg = gr.Markdown()

        with gr.Column(scale=1):
            gr.Markdown("### Pomodoro 🍅 番茄計時器")
            gr.Markdown("**💡 使用方式：** 從下拉選單**選擇**任務，點擊 [▶️ 開始工作] 或 [🍵 開始休息]。計時結束或手動停止後會**自動記錄**到 `pomodoro_logs` 中。")
            timer_area = gr.HTML(TIMER_HTML)  # JS timer UI rendered here
            sel_task = gr.Dropdown(choices=list_task_choices(), label="選擇任務 (開始/結束會記錄)")
            cycles_in = gr.Number(value=1, precision=0, label="番茄數（僅作記錄）")
            pomo_msg = gr.HTML()  # used to display message + returned JS snippets
            with gr.Row():
                start_work_btn = gr.Button("▶️ 開始工作 (25分鐘)")
                end_work_btn = gr.Button("⏹️ 結束工作")
            with gr.Row():
                start_break_btn = gr.Button("🍵 開始休息 (5分鐘)")
                end_break_btn = gr.Button("⏹️ 結束休息")
            gr.HTML("<hr>")
            gr.Markdown("### 統計 📊 數據概覽")
            summary_md = gr.Markdown(today_summary())
            viz_btn = gr.Button("📈 產生統計圖")
            viz_out = gr.Image()
            gr.HTML("<hr>")

    # --- DataFrames 顯示 ---
    gr.Markdown("### 🗂️ 數據表格 (與 Google Sheets 同步)")
    gr.Markdown("**Tasks 列表：** 所有任務數據，可直接在表格內修改（但建議透過上方表單操作以觸發更新邏輯）。")
    tasks_df_out.label = "Tasks 任務列表 (tasks)"

    gr.Markdown("**Logs 列表：** 所有番茄鐘記錄，**只讀**。")
    logs_df_out.label = "Pomodoro Logs 計時記錄 (pomodoro_logs)"

    gr.Markdown("**Clips 列表：** 網頁擷取結果。")
    clips_df_out.label = "Web Clips 網頁擷取結果 (web_clips)"

    # --- 綁定區塊 ---

    # 綁定 add_task 的 inputs (due_in 現在是 gr.Textbox)
    add_btn.click(
        add_task,
        inputs=[task_in, priority_in, est_in, due_in, labels_state, notes_in, planned_in],
        outputs=[add_msg, tasks_df_out, task_choice, sel_task] # 刷新 tasks_df_out 和兩個任務選擇框
    )
    # 修正 update/delete 的輸出，新增 task_choice/sel_task
    update_btn.click(update_task_status, inputs=[task_choice, status_in], outputs=[update_msg, tasks_df_out, task_choice, sel_task])
    delete_btn.click(delete_task, inputs=[task_choice], outputs=[update_msg, tasks_df_out, task_choice, sel_task])

    # Pomodoro: 結束工作後，刷新 tasks_df_out, summary_md, task_choice, sel_task
    end_work_btn.click(
        end_phase,
        inputs=[sel_task, gr.Textbox(value="")],
        outputs=[pomo_msg, pomo_msg]
    ).then(
        _refresh_all,
        outputs=[tasks_df_out, logs_df_out, clips_df_out, task_choice, summary_md, sel_task]
    )
    start_work_btn.click(start_phase, inputs=[sel_task, gr.State("work"), cycles_in], outputs=[pomo_msg, pomo_msg])
    start_break_btn.click(start_phase, inputs=[sel_task, gr.State("break"), cycles_in], outputs=[pomo_msg, pomo_msg])
    end_break_btn.click(end_phase, inputs=[sel_task, gr.Textbox(value="")], outputs=[pomo_msg, pomo_msg])


    # Crawler 綁定修正: 輸出 Dataframe 實例
    crawl_btn.click(
        lambda u,s,m,l: crawl(u,s,m,l),
        inputs=[url_in, selector_in, mode_in, limit_in],
        outputs=[clips_df_out, crawl_msg]
    )
    # add_clips_btn 修正輸出，新增 task_choice/sel_task
    add_clips_btn.click(
        lambda ids,pr,est: add_clips_as_tasks(ids, pr, est), # 直接傳入 ids 字串，讓後端處理
        inputs=[clip_ids_in, priority_in, est_in],
        outputs=[add_clips_msg, clips_df_out, tasks_df_out, task_choice, sel_task] # 輸出 message, clips_df, tasks_df, 任務選擇框
    )

    # Export/Import 綁定修正: 輸出 Dataframe 實例
    import_btn.click(import_tasks_file, inputs=[import_file], outputs=[import_msg, tasks_df_out])
    export_csv_btn.click(lambda: export_tasks_csv(), outputs=[file_out])
    export_json_btn.click(lambda: export_tasks_json(), outputs=[file_out])

    viz_btn.click(generate_stats_image, outputs=[viz_out])

    # Demo load: 修正 _refresh_all 輸出，確保順序與元件實例對應
    demo.load(_refresh_all, outputs=[
        tasks_df_out,
        logs_df_out,
        clips_df_out,
        task_choice,
        summary_md,
        sel_task
    ])

    demo.launch(debug=True)

Google Sheets authentication successful.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d29cbf1c6b0573def3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/gradio/components/dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: [('[todo] (P:H) 寫報告 — 3dc66a45', '3dc66a45'), ('[todo] (P:H) 寫報告 — 134917c8', '134917c8')] or set allow_custom_value=True.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/components/dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: [('[todo] (P:H) 寫報告 — 3dc66a45', '3dc66a45'), ('[todo] (P:H) 寫報告 — 134917c8', '134917c8')] or set allow_custom_value=True.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/components/dropdown.py:230: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: [('[todo] (P:H) 寫報告 — 3dc66a45', '3dc66a45'), ('[todo] (P:H) 寫報告 — 134917c8', '134917c8')] or set allow_custom_value=

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://d29cbf1c6b0573def3.gradio.live
